In [21]:
from tqdm import tqdm
import re
import os
import numpy as np
import argparse
import json
import random
import jieba
import hanlp
from datetime import datetime
#从自写工具包中导入：修饰词判定、修饰词查找
#from utils import modifier_judge, modifier_search, modifier_search_for_mode2

In [22]:
Hanlp = hanlp.load(hanlp.pretrained.mtl.CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_BASE_ZH)
Dep = hanlp.load(hanlp.pretrained.dep.CTB9_DEP_ELECTRA_SMALL)
TOK = hanlp.load( hanlp.pretrained.tok.CTB9_TOK_ELECTRA_BASE)
POS = hanlp.load(hanlp.pretrained.pos.CTB9_POS_ELECTRA_SMALL)


    

In [23]:
#“就业再就业”变成“就业和再就业”
#"抓住....的机遇"要删去；“是把”开头变成“把”；“数字开头要注意”



def get_stopwords_list():
    stopwords = [line.strip() for line in open('../hit_stopwords.txt',encoding='UTF-8').readlines()]
    return stopwords



stopwords = get_stopwords_list()
#print(get_stopwords_list())

wrong_list = ["答卷","成绩单", "底色", "胡锦涛", "叫响", "蓝图", "排头兵","习近平","新篇章", "金字招牌","局面", "之美",
              "让人民满意","难忘", "一盘棋", "一步一个脚印", "一年干", "总书记", "党中央", "党委", "感情", 
              "勇于", "姿态","生根", "真脱贫", "脱真贫","量力而行", "声音", "大旗", "尽力而为", "决胜", "新境界",
              "精神境界", "紧日子", "全心全意", "心上", "主旋律", "压舱石", "中国故事", "谱写", "书写", "文章","民生之", 
              "久久为功", "组合拳", "攻坚克难", "四梁八柱", "脚踏实地", "决胜", "保卫战", "打攻坚战", "之花", "蛋糕", "氛围", 
              "轨道",  "高潮", "敞开", "保驾护航", "以人民为中心", "为代价", "店小二"
              ]  
              #"脚印", "凝聚", "共识", "汇集", "智慧", "口袋", "坚定不移", "护航", "沃土", "先锋", "桥头堡", "领头雁",


def find_matching_element(lst, s):
    for item in lst:
        if item in s:
            return item
    return None

#if not find_matching_element(wrong_list, "努力形成全社会关心困难和下岗职业生活的氛围"):
print(find_matching_element(wrong_list, "努力形成全社会关心困难和下岗职业生活的氛围"))

# 中文数字到整数的映射
chinese_numbers = ['一', '二', '三', '四', '五', '六', '七', '八', '九','十','是','要','我','们','&', '、', '（', '）',
                   ')', '(']
chinese_numbers_true = ['一', '二', '三', '四', '五', '六', '七', '八', '九','十']
measure_word_list = ["批", "产", "张"]

#删去没有用的词，考虑“使？”
strings_to_remove = ["继续", "切实", "不断","必须" ,"一如既往地", "一如既往", "花大力气", "下大力气", "下大气力", "进一步",
                     "驰而不息地", "以更大决心和勇气", "集中力量", "以更大力度", "以更大的力度", "三大战役",
                     "全力以赴", "充满感情、全力以赴地去解决", "千方百计","下大力", "大胆", "聚焦聚力", "加大力度",
                     "举全区之力", "举全省之力", "举全市之力", "持之以恒地", "歼灭战", "大会战", "人民战争", "齐心协力",
                     "集中兵力", "攻坚战持久战", "集中精力", "重拳", "这场硬仗", "五场战役", "重点战役", "两个攻坚战", 
                     "积极创造条件", "一定要", '毫不动摇地', '毫不松懈地', "实现新跨越", "实现新的跨越", 
                     "迈出实质性步伐", "迈出重大步伐", "迈出新步伐", "迈出更大步伐", "迈出实质性步伐", "迈出重要步伐",
                     "迈出大的步伐", "迈出较大步伐", '迈出新的步伐', "迈出更大的步伐", "迈出坚实步伐",
                     "取得新进展", "取得重大进展", "取得突破性进展", "取得新的进展", "取得重要进展", "取得实质性进展",
                     "取得更大进展", "取得较大进展", "取得明显进展", "取得新的更大进展",
                     "取得新突破", "取得重大突破", "取得新的突破", "取得实质性突破",
                     "实现更大作为", "展现更大作为"]


patterns_leading = ["重点要", "重点是", "尤其要", "尤其是", "特别是", "特别要", "重中之重是", "重要的是", "重点在于", "关键在于", 
                    "毫不动摇", "毫不松懈", "驰而不息", "随着", "下功夫","以绣花功夫","一定要", "想方设法", "理直气壮", 
                    "下决心", "更大力度", "持之以恒", "创造条件", "积极", "逐步", 
                    "1、", "2、", "3、", "4、", "5、", "6、", "7、", "8、", "9、"]

patterns_ending = ["力度", "持久战", "、", "会战", "硬仗", "战役", "等", "四大","五大", "三大", "十大", "攻坚战", "进程",
                   "步伐", "的", "新跨越", "建设"]###"建设"!!!!!!!!!!!!!!

Rang_leading = ["绝不", "真正", "努力", "不"]

#让、努力让

def remove_strings(text, strings_to_remove):
    # 将字符串列表转换为正则表达式模式
    pattern = '|'.join(re.escape(s) for s in strings_to_remove)
    # 使用正则表达式模式进行匹配并删除
    return re.sub(pattern, '', text)

def remove_leading_matches(string, patterns):
    # 将模式组合成正则表达式
    pattern = '|'.join(re.escape(p) for p in patterns)
    # 使用正则表达式匹配开头，并将其替换为空字符串
    return re.sub('^(' + pattern + ')', '', string)

def remove_ending_matches(string, patterns):
    # 将模式组合成正则表达式
    pattern = '|'.join(re.escape(p) for p in patterns)
    # 使用正则表达式匹配结尾，并将其替换为空字符串patterns_ending
    return re.sub('(' + pattern + ')$', '', string)


def extract_substrings(text, start, end):
    pattern = rf'{start}(.*?){end}'
    matches = re.findall(pattern, text)
    return matches[0]

def move_stopwords(sentence_list, stopwords_list):
    # 去停用词
    out_list = []
    for word in sentence_list:
        if word not in stopwords_list:
            if word != '\t':
                out_list.append(word)
    return out_list


def RangClean(l):
    test_l = remove_leading_matches(l, Rang_leading)
    if test_l.startswith('让'):
        return ''
    else:
        return l
    
def yiClean(l):
    de_judge = False
    if l.startswith("坚持以"):
        l = l.replace('坚持以', '以')
    if l.startswith("始终坚持以"):
        l = l.replace('始终坚持以', '以')
    if l.startswith("以"):
        if any(l.endswith(substr) for substr in ["为主"]):
            l = '推动'+l[1:-2]
            de_judge = True
        if any(l.endswith(substr) for substr in ["为重点", "为载体", "为基础", "为目标", "为依托", "为龙头", "为纽带",
                                                 "为动力", "为契机", "为指导", "为牵引", "为支撑", "为平台", "为统领",
                                                 "为主体", "为导向", "为引领", "为方向", "为根本", "为抓手", "为支点",
                                                 "为杠杆", "为主题", "为路径", "为牵动", "为带动", "为中心", "为目的",
                                                 "为主线"]):
            l = '推动'+l[1:-3]
            de_judge = True
        if any(l.endswith(substr) for substr in ["为突破口", "为着力点", "为落脚点", "为切入点", "为总抓手", "为总目标"]):
            l = '推动'+l[1:-4]
            de_judge = True
        if any(l.endswith(substr) for substr in ["为主要模式", "为战略基点", "为中心环节", "为重要抓手", "为主攻方向",
                                                 "为战略指引", "为关键抓手", "为战略重点", "为主要措施", "为重要载体"]):
            l = '推动'+l[1:-5]
            de_judge = True
    if de_judge and len(l) < 5:
        return ''
    return l
    
def zaiClean(l):
    if '在' in l:
        l = l.replace('努力在', '在')
        l = l.replace('力争在', '在')
        l = l.replace('力求在', '在')
        l = l.replace('务必在', '在')
        if l.endswith("前提下"):
            l = l[1:-3]
            l = l.rstrip('的')
        if l.endswith("基础上"):
            l = l[1:-3]
            l = l.rstrip('的')
        if l.endswith("上下功夫"):
            l = '推动'+l[1:-4]
        if l.endswith("方面"):
            l = '推动'+l[1:-2]
        if l.endswith("上"):
            l = '推动'+l[1:-1]
        if "上有新" in l:
            l = l.split("上有新")[0]
            l = '推动' + l[1:]
    if l.endswith('方面'):
        l = l[1:-2]
    return l
    
    #if "上" in l:

    
def basicClean(l):
    l = l.replace('\n', '')
    l = l.replace('—', '')
    l = l.replace('•', '')
    l = l.replace('‰', '%')
    l = l.replace('%左右', '%')
    l = l.replace('%以内', '%')
    l = l.replace('牛鼻子', '重点')
    l = l.replace('主战场', '重点')
    l = l.replace('有机结合', '结合')
    
    patternH = r'\[(\d+)\]'
    l = re.sub(patternH, '', l)
    l = remove_strings(l, strings_to_remove)
    
    if l.startswith('在') and l.endswith("的同时"):
        l = l.lstrip('在')
        l = l[:-3]
        
    
    if '路子' in l:
        l = l.replace('路子', '道路')

    pattern = r"像.*?一样"
    pattern2 = r"大兴.*之风"
    matches = re.findall(pattern, l)
    for match in matches:
        #print(match)
        l = l.replace(match, '')
    
    matches2 = re.findall(pattern2, l)
    for match2 in matches2:
        l = l.replace(match2, '')
    
    if l:
        while l[0] in chinese_numbers and len(l)>2:
            if l[0] in chinese_numbers_true:
                if l[1] not in measure_word_list:
                    l = l.lstrip(l[0])
                else:
                    break
            else:
                l = l.lstrip(l[0])

                
        l = remove_leading_matches(l, patterns_leading)
        l = remove_leading_matches(l, patterns_leading)
        if l.startswith('为') and l.endswith("创造条件"):
            l = l.lstrip('为')
            l = l[:-4]
        
        if l.startswith('加快') and l.endswith("步伐"):
            l = l[:-2]
            l = l.lstrip('的')
        
        while l != remove_ending_matches(l, patterns_ending):   
            l = remove_ending_matches(l, patterns_ending)
        #l = remove_ending_matches(l, patterns_ending)
        if '腾笼换鸟' in l:
            l = l.replace( '腾笼换鸟', '转型升级')
            if '、凤凰涅槃' in l:
                l = l.replace( '凤凰涅槃', '')
        #    l = extract_substrings(l, '打好', '攻坚战')
        #    l = '推进'+l
        #    l = l.replace('两个', '')
        #    l = l.replace('三个', '')
        return l
    else:
        return ''


ll = '推动城市化建设'
if basicClean(ll):
    print(basicClean(ll))
    print('111')
    fix_ll = RangClean(basicClean(ll))
    print(fix_ll)
    #print(fix_ll[-3:])
    if '在' in fix_ll:
        fix_ll = zaiClean(fix_ll)
        print(fix_ll)
        #print(fix_ll[-3:])
    if '以' in fix_ll:
        fix_ll = yiClean(fix_ll)
        print(fix_ll)
    
        
        
    #print(zaiClean(basicClean(ll)))



氛围
推动城市化
111
推动城市化


In [24]:
def contains_all_elements(list_a, list_b):
    return set(list_b).issubset(set(list_a))

def remove_duplicates(lst):
    return [x for i, x in enumerate(lst) if x not in lst[:i]]

def modifier_judge(index, dependency):
    modifyJudge = False
    for i1 in dependency:
        if i1[1] in ['nn', 'prep', 'nummod', 'amod','mmod', 'assmod','vmod',  'advmod', 'dep', 'clf','nsubj','rcmod'] and i1[0]-1 == index:
            modifyJudge = True
        if i1[1] == 'range' and i1[0]-1 == dependency[index][0]-1:
            modifyJudge = True    
    return modifyJudge

def modifier_judge_conj(index, dependency):
    modifyJudge = False
    for i1 in dependency:
        if i1[1] in ['nn', 'prep', 'nummod', 'amod','mmod', 'assmod','vmod', 'advmod', 'dep', 'clf','nsubj', 'dobj', 'rcmod'] and i1[0]-1 == index:
            modifyJudge = True
        if i1[1] == 'range' and i1[0]-1 == dependency[index][0]-1:
            modifyJudge = True    
    return modifyJudge


def modifier_search(index, seg_list, dependency):
    phrase=[]
    phrase.append(seg_list[dependency[index][0]-1])
    #multiple_key = False
    #key.append(seg_list[index])
    #for index1, i1 in enumerate(dependency):
    #    if i1[1] in ['conj'] and seg_list[i1[0]-1] == seg_list[index]: 
    #        key.append(seg_list[index1])
    phrase.append(seg_list[index])
    for index2, i2 in enumerate(dependency):
        if i2[1] in ['nn', 'prep', 'nummod', 'amod', 'assmod','vmod', 'advmod', 'dep','mmod', 'clf','rcmod'] and i2[0]-1 == index:
            if seg_list[index2] not in phrase:
                phrase.append(seg_list[index2])
            for index2_1, i2_1 in enumerate(dependency):
                if i2_1[1] in ['nn', 'nummod', 'amod', 'assmod','vmod', 'dep', 'advmod', 'clf','rcomp'] and i2_1[0]-1 == index2:
                    if seg_list[index2_1] not in phrase:
                        phrase.append(seg_list[index2_1])
                    for index2_2, i2_2 in enumerate(dependency):
                        if i2_2[1] in ['nn', 'nummod', 'amod', 'assmod','vmod', 'advmod','rcomp'] and i2_2[0]-1 == index2_1:
                            if seg_list[index2_2] not in phrase:
                                phrase.append(seg_list[index2_2])
                            for index2_4, i2_4 in enumerate(dependency):
                                if i2_4[1] in ['nn', 'nummod', 'amod', 'assmod','vmod', 'advmod','rcomp'] and i2_4[0]-1 == index2_2:
                                    if seg_list[index2_4] not in phrase:
                                        phrase.append(seg_list[index2_4])
#                                    for index2_5, i2_5 in enumerate(dependency):
#                                        if i2_5[1] in ['nn', 'prep', 'nummod', 'amod', 'assmod','vmod', 'advmod', 'clf','nsubj'] and i2_5[0]-1 == index2_4:
#                                            phrase.append(seg_list[index2_5])                            
        if i2[1] == 'range' and i2[0]-1 == dependency[index][0]-1:
            for index2_3, i2_3 in enumerate(dependency):
                if i2_3[1] == 'nummod' and i2_3[0]-1 == index2_3:
                    if seg_list[index2] not in phrase and seg_list[index2_3] not in phrase:
                        phrase.append(seg_list[index2])
                        phrase.append(seg_list[index2_3])

                
    return phrase

def modifier_search_for_mode2(index, seg_list, dependency):
    phrase=[]
    #multiple_key = False
    #key.append(seg_list[index])
    #for index1, i1 in enumerate(dependency):
    #    if i1[1] in ['conj'] and seg_list[i1[0]-1] == seg_list[index]: 
    #        key.append(seg_list[index1])
    phrase.append(seg_list[index])
    for index2, i2 in enumerate(dependency):
        if i2[1] in ['nn', 'prep', 'nummod', 'amod', 'assmod','vmod', 'range', 'mmod','advmod','rcmod', 'dep'] and i2[0]-1 == index:
            if seg_list[index2] not in phrase:
                phrase.append(seg_list[index2])
            for index2_1, i2_1 in enumerate(dependency):
                if i2_1[1] in ['nn', 'prep', 'nummod', 'amod', 'assmod','vmod', 'advmod','dep'] and i2_1[0]-1 == index2:
                    if seg_list[index2_1] not in phrase:
                        phrase.append(seg_list[index2_1])
    return phrase

def modifier_search_for_mode2_conj(index, seg_list, dependency):
    phrase=[]
    #multiple_key = False
    #key.append(seg_list[index])
    #for index1, i1 in enumerate(dependency):
    #    if i1[1] in ['conj'] and seg_list[i1[0]-1] == seg_list[index]: 
    #        key.append(seg_list[index1])
    phrase.append(seg_list[index])
    for index2, i2 in enumerate(dependency):
        if i2[1] in ['nn', 'nsubj', 'dobj', 'prep', 'nummod', 'amod', 'assmod','vmod', 'range', 'mmod','advmod','rcmod', 'dep'] and i2[0]-1 == index:
            if seg_list[index2] not in phrase:
                phrase.append(seg_list[index2])
            for index2_1, i2_1 in enumerate(dependency):
                if i2_1[1] in ['nn', 'prep', 'nummod', 'amod', 'assmod','vmod', 'advmod','dep'] and i2_1[0]-1 == index2:
                    if seg_list[index2_1] not in phrase:
                        phrase.append(seg_list[index2_1])
    return phrase

def read_time_list(path):
    list_time=[]
    with open(path, 'r') as time_read:
        for line in time_read:
            times = line.split(',')
            times.remove('\n')
            list_time.append(times)
    return list_time


def phrase2vec(lines):
    phrases_list = []
    for  phrases in lines:
        if phrases[0] in vectors:
            array = vectors[phrases[0]].reshape(1,300)
        else:
            print('word can not find')
        for i in range(1,len(phrases)):
            if phrases[i] in vectors:
                array = np.append(array, vectors[phrases[i]].reshape(1,300),axis = 0)
            else:
                print('word can not find')
        phrases_list.append(array)
    return phrases_list

def phrase_extraction(sentence, sentenceO):
    seg_list = TOK([sentence][0])
    seg_list = move_stopwords(seg_list, stopwords)
    if seg_list:
        position = POS([seg_list], conll=False)[0]
        dependency = Dep([seg_list], conll=False)[0]
        #print(seg_list)
        #print(position)
        #print(dependency)
        list1 = []
        list2 = []
        keys = []
    #list2 = 
        for index, i in enumerate(dependency):
            #复合名词修饰、介词性修饰语、数词修饰语、形容词修饰语、关联修饰
            #if i[1] in ['nn', 'prep', 'nummod', 'amod', 'assmod']:
            #'ccomp',
            if i[1] in ['dobj', 'pobj','lobj','nsubj'] or (i[1] == 'ccomp' and position[index] == 'NN'):
                mode1 = False
                if position[i[0]-1] in ['VV', 'VE', 'VC','P'] and dependency[i[0]-1][1] not in ['prep','nn', 'nummod', 'amod', 'assmod','vmod', 'advmod', 'rcmod']:
                    if seg_list[index] not in keys:
                        if modifier_judge(index, dependency) == True:
                            #print(seg_list[index])
                            keys.append(seg_list[index])
                            list1.append(modifier_search(index, seg_list, dependency))
                        else:
                            if i[1] in ['dobj', 'pobj','lobj','nsubj']:
                                keys.append(seg_list[index])
                                list1.append([seg_list[i[0]-1],seg_list[index]])
                else:
                    if modifier_judge(index, dependency) == True:
                        list2.append(modifier_search_for_mode2(index, seg_list, dependency))
                    else:
                        list2.append([seg_list[index]])
        for index2, i2 in enumerate(dependency):        
            if i2[1] =='conj':
                if position[index2] not in ['VV', 'VE', 'VC','P']:
                    if modifier_judge_conj(index2, dependency) == True:
                        if modifier_search_for_mode2_conj(index2, seg_list, dependency) not in list2:
                            list2.append(modifier_search_for_mode2_conj(index2, seg_list, dependency))
                    else:
                        if [seg_list[index2]] not in list2:
                            list2.append([seg_list[index2]])
                else:
                    check_conj = True
                    for check_list1 in list1:
                        if seg_list[index2] in check_list1:
                            check_conj = False
                    if check_conj:
                        if modifier_judge(index2, dependency) == True:
                            if modifier_search_for_mode2(index2, seg_list, dependency) not in list2:
                                list2.append(modifier_search_for_mode2(index2, seg_list, dependency))
                        else:
                            if [seg_list[index2]] not in list2:
                                list2.append([seg_list[index2]])
            if i2[1] == 'root' and position[index2] in ['NN', 'NT', 'NR','M']:
                if modifier_judge(index, dependency) == True:
                    list2.append(modifier_search_for_mode2(index, seg_list, dependency))
            if i2[1] == 'range' and position[i2[0]-1] in ['VV', 'VE', 'VC','P']:
                if modifier_judge(index2, dependency) == True:
                    list2.append(modifier_search_for_mode2(index2, seg_list, dependency))
            #if i2[1] == 'ccomp':
            #    if modifier_judge(index2, dependency) == True:
            #        list2.append(modifier_search_for_mode2(index2, seg_list, dependency))
        for key2 in list2:
            if key2[0] not in keys:
                keys.append(key2[0])
                
        # 传统方式，使用列表推导式
        #print(list1)
        #print(list2)
        keys = remove_duplicates(keys)
        list2 = remove_duplicates(list2)
        
        for xOfList1 in list1:
            for xOfList2 in list2:
                if contains_all_elements(xOfList1[1:], xOfList2):
                    list2.remove(xOfList2)

        # 简写方式，使用 range
        squares = [x**2 for x in range(10)]
        return keys, list1, list2, sentence, sentenceO
    else:
        return ['']
    
    

def textModification(path):
    num_text = 0
    len_raw = 0
        
    with open(path, encoding='utf-8', errors='ignore') as f:
        the_line = False
        the_time = True
        lines1=[]
        lines2=[]
        for line in f:
            #line = line.replace(' ','')
            #line = line.replace('　','')
            line="".join(line.split())
            ret = re.findall(r'(\d{4})\s*[\./年-]\s*(\d{1,2})\s*[\./月-]\s*(\d{1,2})\s*日?', line)
            #print(line)
            if ret and the_time:
                year, month, day = ret[0]
                #time = '{}/{}/{}'.format(year, month.lstrip(), day.lstrip())
                time = datetime(int(year), int(month.lstrip()), int(day.lstrip()),hour=0,minute=0,second=0,microsecond=0)
                time = time.strftime("%Y-%m-%d")
                the_time = False
            if line.startswith('二&、'):
                the_line = True
           #     continue         
            if the_line:
                if line.startswith('\n'):
                    a=1
                #elif line.count('，') >= 5:
                    #lines.append(line)
                    #print(line) 
                #    line = re.split('，|;|。', line)
                 #   for l in line:
                 #       if l != '\n' and len(l) !=0:
                 #           lines.append(l.replace('\n', ''))
                else:
                    line = line.replace('％', '%')
                    line = line.replace('%。左右', '%左右')
                    line = line.replace('%。以内', '%以内')
                    line = line.replace('“', '')
                    line = line.replace('”', '')
                    line = re.split('：|；|;|。|，|！|!|,|包括:', line)
                    for l in line:
                        if l != '\n' and len(l) !=0:
                            len_raw = len_raw+1
                            if find_matching_element(wrong_list, l):
                                continue
                            else:
                                l_c = basicClean(l)
                                if l_c:
                                    l_c = RangClean(basicClean(l_c))
                                    if '在' in l_c:
                                        l_c = zaiClean(l_c)
                                    if '以' in l_c:
                                        l_c = yiClean(l_c)
                                    num_text = num_text+len(l_c)
                                    phrases = phrase_extraction(l_c, l)
                                    if phrases[0]:
                                        lines2.append(phrases) 
                                    else:
                                        cccccc=1
                                    #print(l)
    tuple1=tuple(lines2)
    tuple1_summarize=tuple(lines1)
    return tuple1, time, len_raw, num_text, tuple1_summarize

In [25]:
T = TOK(['加强以全科医生为重点的基层医疗卫生队伍建设'][0])

print(T)
#print(111)



#doc = HanLP(['新增可再生能源和原料用能不纳入能源消费总量控制'], tasks=['pos'])

ccc = POS([T], conll=False)[0]
#print(phrase_extraction('促进三大产业协调发展'))银行决定先取得信用评级
dddd = Dep([T], conll=False)[0]
print(dddd)
print(ccc)


l = ' 加强以全科医生为重点的基层医疗卫生队伍建设'
print(find_matching_element(wrong_list, l))
l = basicClean(l)
l = zaiClean(l)
l = yiClean(l)
print(phrase_extraction(l, '更加奋发有为地推进发展'))

#已有动词不能再当关键词，考虑删去更小的集合
#print(phrase_extraction(l, '更加奋发有为地推进发展'))


#以“五个一批”企业和“八个一百”工程为重点, 实现“二次创新”
##和-以及：推进旅游、现代物流、金融保险以及社区服务等产业的发展；以“五个一批”企业和“八个一百”工程为重点。以“五个一批”企业和“八个一百”工程为重点


['加强', '以', '全科', '医生', '为', '重点', '的', '基层', '医疗', '卫生', '队伍', '建设']
[(0, 'root'), (5, 'prep'), (4, 'nn'), (2, 'pobj'), (12, 'rcmod'), (5, 'dobj'), (5, 'cpm'), (11, 'nn'), (11, 'nn'), (11, 'nn'), (12, 'nn'), (1, 'dobj')]
['VV', 'P', 'NN', 'NN', 'VV', 'NN', 'DEC', 'NN', 'NN', 'NN', 'NN', 'NN']
None
(['队伍', '医生', '重点'], [['加强', '队伍', '为', '基层', '医疗', '卫生']], [['医生', '全科'], ['重点']], ' 加强以全科医生为重点的基层医疗卫生队伍', '更加奋发有为地推进发展')


In [26]:
file_name_list = os.listdir('../new/policies_cleaned&undivided/reports_clean_0520_withoutG')
file_name_list.remove('.DS_Store')
#file_name_list.remove('国务院')
print(file_name_list)

#tatal_wrong_list = ["试点"]  
#"要" “为”在开头,每段最后一句或者第一句的引号,"以"开头; 在。。。上取得重大突破

#wrong_dict = {j:[] for j in tatal_wrong_list}
#print(wrong_dict)

['安徽省', '山西省', '福建省', '上海市', '陕西省', '广西壮族自治区', '宁夏回族自治区', '重庆市', '浙江省', '贵州省', '湖北省', '湖南省', '青海省', '黑龙江省', '河北省', '河南省', '江西省', '天津市', '国务院', '辽宁省', '江苏省', '新疆维吾尔自治区', '山东省', '甘肃省', '四川省', '海南省', '内蒙古自治区', '北京市', '云南省', '西藏自治区', '吉林省', '广东省']


In [35]:


for year in range(1998,2025):
    for p in file_name_list[28:]:
        print(p+" ", year)
        text_path = "../new/policies_cleaned&undivided/reports_clean_0520_withoutG/"+p+"/"+p+str(year)+"年政府工作报告.txt"
        save_path_forward = "../new/policies_cleaned&undivided/raw_extraction241101/"+p+str(year)+"年政府工作报告.txt"
        if os.path.exists(text_path):
            lines_forward, time, raw_len, text_num, lines_summarize = textModification(text_path)
            with open(file =save_path_forward, mode='w',encoding='utf - 8') as f_forward:
                f_forward.write('提取短语数 '+str(len(lines_forward))+'，全部短语数 '+str(raw_len)+'，字数 '+str(text_num)+'\n')
                #print(lines_forward[0])
                #print(lines_forward[0][0],lines_forward[0][1],lines_forward[0][2],lines_forward[0][3])
                for index, every_line in enumerate(lines_forward):
                    f_forward.write(p+str(year)+' '+str(index)+',')
                    for index_key, key in enumerate(every_line[0]):
                        f_forward.write(key)
                        if index_key < len(every_line[0])-1:
                            f_forward.write(' ')
                    f_forward.write(',')
                    for index_mode1_phrase, mode1_phrase in enumerate(every_line[1]):
                        for index_mode1_word, mode1_word in enumerate(mode1_phrase):
                            f_forward.write(mode1_word)
                            if index_mode1_word < len(mode1_phrase)-1:
                                f_forward.write(' ')
                        if index_mode1_phrase< len(every_line[1])-1:
                            f_forward.write(';')
                    f_forward.write(',')
                    for index_mode2_phrase, mode2_phrase in enumerate(every_line[2]):
                        for index_mode2_word, mode2_word in enumerate(mode2_phrase):
                            f_forward.write(mode2_word)
                            if index_mode2_word < len(mode2_phrase)-1:
                                f_forward.write(' ')
                        if index_mode2_phrase< len(every_line[2])-1:
                            f_forward.write(';')
                    f_forward.write('。')
                    f_forward.write(every_line[3])
                    f_forward.write('。')
                    f_forward.write(every_line[4]) 
                    f_forward.write('\n')

        else:
            print(111)


云南省  1998
西藏自治区  1998
111
吉林省  1998
广东省  1998
云南省  1999
西藏自治区  1999
111
吉林省  1999
广东省  1999
云南省  2000
西藏自治区  2000
111
吉林省  2000
广东省  2000
云南省  2001
西藏自治区  2001
111
吉林省  2001
广东省  2001
云南省  2002
西藏自治区  2002
吉林省  2002
广东省  2002
云南省  2003
西藏自治区  2003
吉林省  2003
广东省  2003
云南省  2004
西藏自治区  2004
吉林省  2004
广东省  2004
云南省  2005
西藏自治区  2005
吉林省  2005
广东省  2005
云南省  2006
西藏自治区  2006
吉林省  2006
广东省  2006
云南省  2007
西藏自治区  2007
吉林省  2007
广东省  2007
云南省  2008
西藏自治区  2008
吉林省  2008
广东省  2008
云南省  2009
西藏自治区  2009
吉林省  2009
广东省  2009
云南省  2010
西藏自治区  2010
吉林省  2010
广东省  2010
云南省  2011
西藏自治区  2011
吉林省  2011
广东省  2011
云南省  2012
西藏自治区  2012
吉林省  2012
广东省  2012
云南省  2013
西藏自治区  2013
吉林省  2013
广东省  2013
云南省  2014
西藏自治区  2014
吉林省  2014
广东省  2014
云南省  2015
西藏自治区  2015
吉林省  2015
广东省  2015
云南省  2016
西藏自治区  2016
吉林省  2016
广东省  2016
云南省  2017
西藏自治区  2017
吉林省  2017
广东省  2017
云南省  2018
西藏自治区  2018
吉林省  2018
广东省  2018
云南省  2019
西藏自治区  2019
吉林省  2019
广东省  2019
云南省  2020
西藏自治区  2020
吉林省  2020
广东省  2020
云南省  2021
西藏自治区  2

In [239]:
print(9+3+2+2+2+2+2+4)

26


In [20]:
NR_list= []

wrong_list2 = ["“", "‘"]

list_candidate = []
list_candidate2 = []
list_candidate3 = []

def find_first_matching_element(lst, s):
    for item in lst:
        if item in s:
            return item
    return None

for year in range(2003,2025):
    for p in file_name_list:
        #print(p+" ", year)
        text_path = "../new/policies_cleaned&undivided/reports_clean_0520_withoutG/"+p+"/"+p+str(year)+"年政府工作报告.txt"
        save_path_forward = "../new/test/"+p+str(year)+"年政府工作报告.txt"
        the_line = False
        the_time = True
        lines1=[]
        lines2_candidate=[]
        with open(text_path, encoding='utf-8', errors='ignore') as f:
            for line in f:
                line="".join(line.split())
                ret = re.findall(r'(\d{4})\s*[\./年-]\s*(\d{1,2})\s*[\./月-]\s*(\d{1,2})\s*日?', line)
            #print(line)
                if ret and the_time:
                    year, month, day = ret[0]
                #time = '{}/{}/{}'.format(year, month.lstrip(), day.lstrip())
                    time = datetime(int(year), int(month.lstrip()), int(day.lstrip()),hour=0,minute=0,second=0,microsecond=0)
                    time = time.strftime("%Y-%m-%d")
                    the_time = False
                if line.startswith('二&、'):
                    the_line = True
           #     continue         
                if the_line:
                    if line.startswith('\n'):
                        a=1
                #elif line.count('，') >= 5:
                    #lines.append(line)
                    #print(line) 
                #    line = re.split('，|;|。', line)
                 #   for l in line:
                 #       if l != '\n' and len(l) !=0:
                 #           lines.append(l.replace('\n', ''))
                    else:
                        line = line.replace('％', '%')
                        line = line.replace('%。左右', '%左右')
                        line = line.replace('%。以内', '%以内')
                        line_set = re.split('：|；|;|。|，|,|！|包括:', line)
                        for l in line_set:
                            if l != '\n' and len(l) !=0:
                                l_c = basicClean(l)
                                if l_c:
                                    if find_first_matching_element(tatal_wrong_list, l_c):
                                        wrong_dict[find_first_matching_element(tatal_wrong_list, l_c)].append([l, p+str(year)])
                                    #if '把' in l_c: #and '上海' not in l_c:
                                    #    list_candidate2.append([l, p+str(year)])
                                        
                                    #seg_list = TOK([l_c][0])
                                    #seg_list = move_stopwords(seg_list, stopwords)
                                    #if seg_list:
                                        #position = POS([seg_list], conll=False)[0]
                                        #for index, p in enumerate(position):
                                            #if p == 'NR' and seg_list[index] not in NR_list:
                                                #NR_list.append(seg_list[index])
                                                

                                    #if '让' in l_c and '出让' not in l_c and '转让' not in l_c and '让出' not in l_c:
                                    #    list_candidate3.append([l_c, p+str(year)])


for i in tatal_wrong_list:
    if wrong_dict[i] != []:
        with open("../new/test/"+i+"政府工作报告.txt", mode='w',encoding='utf - 8') as f:
            for every_line in wrong_dict[i]:
                #print(every_line)
                f.write(every_line[0])
                f.write(';  ')
                f.write(every_line[1])
                f.write('\n')
        
#with open("../new/test/把政府工作报告.txt", mode='w',encoding='utf - 8') as f2:
#    for every_line2 in list_candidate2:
        #print(every_line)
#        f2.write(every_line2[0])
#        f2.write(';  ')
#        f2.write(every_line2[1])
#        f2.write('\n')
        
#with open("../new/test/政府工作报告ex.txt", mode='w',encoding='utf - 8') as f3:
#    for every_line3 in list_candidate3:
        #print(every_line)
#        f3.write(every_line3[0])
#        f3.write(';  ')
#        f3.write(every_line3[1])
#        f3.write('\n')

In [18]:
file_name_list = os.listdir('province_annual_data')
print(file_name_list)
time_list = []
for year in range(2002,2023):
    year_list = []
    for p in file_name_list:
        #print(p+" ", year)
        text_path = "province_annual_data/"+p+"/"+p+str(year)+"年政府工作报告.txt"
        with open(text_path, encoding='utf-8', errors='ignore') as f:
            the_time = True
            for line in f:
                line="".join(line.split())
                ret = re.findall(r'(\d{4})\s*[\./年-]\s*(\d{1,2})\s*[\./月-]\s*(\d{1,2})\s*日?', line)
                if ret and the_time:
                    year, month, day = ret[0]
                    time = datetime(int(year), int(month.lstrip()), int(day.lstrip()),hour=0,minute=0,second=0,microsecond=0)
                    time = time.strftime("%Y-%m-%d")
                    year_list.append([p, time])
                    break
    time_list.append(sorted(year_list, key=lambda x: x[1], reverse=False))
    print(len(year_list))
print(time_list)
with open(file ='time_table.txt', mode='w',encoding='utf - 8') as t_w:
    for years in time_list:
        for couple in years:
            t_w.write(couple[0])
            t_w.write(' ')
            t_w.write(couple[1])
            t_w.write(',')
        t_w.write('\n')

['上海市', '云南省', '内蒙古自治区', '北京市', '吉林省', '四川省', '国务院', '天津市', '宁夏回族自治区', '安徽省', '山东省', '山西省', '广东省', '广西', '新疆维吾尔自治区', '江苏省', '江西省', '河北省', '河南省', '浙江省', '海南省', '湖北省', '湖南省', '甘肃省', '福建省', '西藏自治区', '贵州省', '辽宁省', '重庆市', '陕西省', '青海省', '黑龙江省']
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
[[['新疆维吾尔自治区', '2002-01-15'], ['贵州省', '2002-01-18'], ['青海省', '2002-01-19'], ['内蒙古自治区', '2002-01-22'], ['云南省', '2002-01-23'], ['山西省', '2002-01-23'], ['福建省', '2002-01-23'], ['安徽省', '2002-01-24'], ['河北省', '2002-01-24'], ['河南省', '2002-01-24'], ['重庆市', '2002-01-24'], ['北京市', '2002-01-25'], ['宁夏回族自治区', '2002-01-25'], ['广西', '2002-01-25'], ['江西省', '2002-01-25'], ['浙江省', '2002-01-25'], ['吉林省', '2002-01-26'], ['甘肃省', '2002-01-26'], ['陕西省', '2002-01-26'], ['广东省', '2002-01-28'], ['湖南省', '2002-01-28'], ['黑龙江省', '2002-01-29'], ['海南省', '2002-02-02'], ['江苏省', '2002-02-20'], ['湖北省', '2002-02-21'], ['上海市', '2002-02-22'], ['辽宁省', '2002-02-22'], ['国务院', '2002-03-05'], ['山东省', '2002-03-21'], ['四川省', '2002-04-

In [17]:

        print(corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))

2022-12-12 16:15:02.468 | DEBUG    | text2vec.sentence_model:__init__:74 - Use device: cpu






Query: 如何更换花呗绑定银行卡

Top 5 most similar sentences in corpus:
花呗更改绑定银行卡 (Score: 0.8551)
我什么时候开通了花呗 (Score: 0.7212)
A man is eating food. (Score: 0.3118)
A man is eating a piece of bread. (Score: 0.2992)
A monkey is playing drums. (Score: 0.2922)




Query: A man is eating pasta.

Top 5 most similar sentences in corpus:
A man is eating food. (Score: 0.7840)
A man is riding a white horse on an enclosed ground. (Score: 0.6906)
A man is eating a piece of bread. (Score: 0.6831)
A man is riding a horse. (Score: 0.6515)
Two men pushed carts through the woods. (Score: 0.5270)




Query: Someone in a gorilla costume is playing a set of drums.

Top 5 most similar sentences in corpus:
A monkey is playing drums. (Score: 0.6758)
A man is riding a white horse on an enclosed ground. (Score: 0.6351)
The girl is carrying a baby. (Score: 0.5438)
A man is riding a horse. (Score: 0.5002)
A man is eating a piece of bread. (Score: 0.4916)




Query: A cheetah chases prey on across a field.

Top 5 most sim